In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms


In [2]:
def get_data(file):
    df=pd.read_csv(file,names=[0,1,2,3,4])
    d={'Iris-setosa':0,'Iris-virginica':2,'Iris-versicolor':1}
    df[4]=df[4].map(d)
    X=df.iloc[:,0:4]
    y=df.iloc[:,4:]
    X=np.array(X)
    y=np.array(y)
    y = np.reshape(y,(-1,1))
    ss=StandardScaler().fit(X)
    X=ss.transform(X)
    X=np.array(X,dtype='float32')
    y=np.array(y,dtype='int32')
    return train_test_split(X,y,test_size=0.7,random_state=0)

In [3]:
X_train,X_test,y_train,y_test=get_data('d.txt')

In [4]:
batch_size = 32
learning_rate = 1e-2
num_epoches = 50

In [5]:
class Neuralnetwork(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(Neuralnetwork, self).__init__()
        self.layer1 = nn.Linear(in_dim, n_hidden_1)
        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.layer3 = nn.Linear(n_hidden_2, out_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [6]:
model = Neuralnetwork(4, 20, 16, 3)
if torch.cuda.is_available():
    model = model.cuda()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
for epoch in range(num_epoches):
    running_loss = 0.0
    running_acc = 0.0
    for i in range(X_train.shape[0]):
        acc = 0.0
        x, y = torch.from_numpy(X_train[i]), torch.from_numpy(y_train[i]).type(torch.LongTensor)
        if torch.cuda.is_available():
            x = Variable(x).cuda()
            y = Variable(y).cuda()
        else:
            x = Variable(x)
            y = Variable(y)
       
        out = model(x).view(1,-1)
        loss = criterion(out, y)
    
        pred = torch.argmax(out)
        correct = (pred==y.data[0])
        
        acc += correct.item()
        running_loss += loss * y.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, num_epoches, running_loss / (batch_size * i),
        acc))
    model.eval()


[1/50] Loss: 0.002087, Acc: 1.000000
[2/50] Loss: 0.002064, Acc: 1.000000
[3/50] Loss: 0.002042, Acc: 1.000000
[4/50] Loss: 0.002021, Acc: 1.000000
[5/50] Loss: 0.002001, Acc: 1.000000
[6/50] Loss: 0.001981, Acc: 1.000000
[7/50] Loss: 0.001963, Acc: 1.000000
[8/50] Loss: 0.001945, Acc: 1.000000
[9/50] Loss: 0.001928, Acc: 1.000000
[10/50] Loss: 0.001911, Acc: 1.000000
[11/50] Loss: 0.001896, Acc: 1.000000
[12/50] Loss: 0.001881, Acc: 1.000000
[13/50] Loss: 0.001866, Acc: 1.000000
[14/50] Loss: 0.001852, Acc: 1.000000
[15/50] Loss: 0.001839, Acc: 1.000000
[16/50] Loss: 0.001826, Acc: 1.000000
[17/50] Loss: 0.001813, Acc: 1.000000
[18/50] Loss: 0.001801, Acc: 1.000000
[19/50] Loss: 0.001789, Acc: 1.000000
[20/50] Loss: 0.001778, Acc: 1.000000
[21/50] Loss: 0.001767, Acc: 1.000000
[22/50] Loss: 0.001757, Acc: 1.000000
[23/50] Loss: 0.001747, Acc: 1.000000
[24/50] Loss: 0.001737, Acc: 1.000000
[25/50] Loss: 0.001728, Acc: 1.000000
[26/50] Loss: 0.001719, Acc: 1.000000
[27/50] Loss: 0.00171

In [13]:
val_acc = 0.0
for i in range(X_test.shape[0]):
    x, y = torch.from_numpy(X_test[i]), torch.from_numpy(y_test[i]).type(torch.LongTensor)
    if torch.cuda.is_available():
        x = Variable(x).cuda()
        y = Variable(y).cuda()
    else:
        x = Variable(x)
        y = Variable(y)

    out = model(x).view(1,-1)
    loss = criterion(out, y)

    pred = torch.argmax(out)
    correct = (pred==y.data[0])
    
    val_acc += correct.item()
    running_loss += loss * label.size(0)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    
print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
    epoch + 1, num_epoches, running_loss / (batch_size * i),
    val_acc))


[50/50] Loss: 0.002896, Acc: 100.000000
